In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81085,6.0,7.0,7.0,4.0,9.0,1.0,3.0,5.0,4.0,7.0,...,5.0,8.0,7.0,2.0,1.0,5.0,1.0,2022-11-26,265,2.0
81086,10.0,5.0,4.0,11.0,8.0,2.0,2.0,5.0,7.0,8.0,...,6.0,2.0,1.0,3.0,2.0,8.0,4.0,2022-11-27,265,3.0
81087,6.0,2.0,2.0,0.0,3.0,7.0,24.0,20.0,10.0,18.0,...,5.0,5.0,4.0,4.0,8.0,1.0,7.0,2022-11-28,265,10.0
81088,4.0,6.0,0.0,1.0,1.0,7.0,22.0,18.0,25.0,17.0,...,10.0,11.0,13.0,3.0,4.0,3.0,1.0,2022-11-29,265,3.0


In [4]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(48495, 674)
y_test.shape=(48495,)


In [5]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [6]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)
best_params = study.best_trial.params
print(f'{best_params=}')

[I 2023-02-26 19:04:23,365] A new study created in memory with name: no-name-4400cc35-89b6-4d0a-90eb-c4dad7712f5e


[LightGBM] [Warning] feature_fraction is set=0.8109926327233727, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8109926327233727
[LightGBM] [Warning] bagging_fraction is set=0.38573231922061996, subsample=1.0 will be ignored. Current value: bagging_fraction=0.38573231922061996
[LightGBM] [Warning] feature_fraction is set=0.8109926327233727, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8109926327233727
[LightGBM] [Warning] bagging_fraction is set=0.38573231922061996, subsample=1.0 will be ignored. Current value: bagging_fraction=0.38573231922061996


[I 2023-02-26 19:04:38,568] Trial 0 finished with value: 3.4309356421306587 and parameters: {'num_leaves': 197, 'feature_fraction': 0.8109926327233727, 'bagging_fraction': 0.38573231922061996, 'min_child_samples': 33}. Best is trial 0 with value: 3.4309356421306587.


[LightGBM] [Warning] feature_fraction is set=0.9805529319999866, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9805529319999866
[LightGBM] [Warning] bagging_fraction is set=0.41008380050111504, subsample=1.0 will be ignored. Current value: bagging_fraction=0.41008380050111504
[LightGBM] [Warning] feature_fraction is set=0.9805529319999866, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9805529319999866
[LightGBM] [Warning] bagging_fraction is set=0.41008380050111504, subsample=1.0 will be ignored. Current value: bagging_fraction=0.41008380050111504


[I 2023-02-26 19:04:53,054] Trial 1 finished with value: 3.2296293804012617 and parameters: {'num_leaves': 204, 'feature_fraction': 0.9805529319999866, 'bagging_fraction': 0.41008380050111504, 'min_child_samples': 16}. Best is trial 1 with value: 3.2296293804012617.


...

[LightGBM] [Warning] feature_fraction is set=0.5162844543976608, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5162844543976608
[LightGBM] [Warning] bagging_fraction is set=0.4995174126880668, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4995174126880668
[LightGBM] [Warning] feature_fraction is set=0.5162844543976608, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5162844543976608
[LightGBM] [Warning] bagging_fraction is set=0.4995174126880668, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4995174126880668


[I 2023-02-26 19:05:08,887] Trial 3 finished with value: 3.649730513497209 and parameters: {'num_leaves': 45, 'feature_fraction': 0.5162844543976608, 'bagging_fraction': 0.4995174126880668, 'min_child_samples': 89}. Best is trial 1 with value: 3.2296293804012617.


[LightGBM] [Warning] feature_fraction is set=0.4520280535030409, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4520280535030409
[LightGBM] [Warning] bagging_fraction is set=0.6082892446095596, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6082892446095596
[LightGBM] [Warning] feature_fraction is set=0.4520280535030409, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4520280535030409
[LightGBM] [Warning] bagging_fraction is set=0.6082892446095596, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6082892446095596


[I 2023-02-26 19:05:13,363] Trial 4 finished with value: 3.528208719677894 and parameters: {'num_leaves': 50, 'feature_fraction': 0.4520280535030409, 'bagging_fraction': 0.6082892446095596, 'min_child_samples': 73}. Best is trial 1 with value: 3.2296293804012617.


best_params={'num_leaves': 204, 'feature_fraction': 0.9805529319999866, 'bagging_fraction': 0.41008380050111504, 'min_child_samples': 16}


In [7]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.9805529319999866, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9805529319999866
[LightGBM] [Warning] bagging_fraction is set=0.41008380050111504, subsample=1.0 will be ignored. Current value: bagging_fraction=0.41008380050111504
test_mae=2.5065


On previous notebooks, we saw that **LightGBM** model returned a test_mae=2.5145 and a test_mae=2.5281 with feature engineering. So, hyperparameter tuning seems to improve our oirginal LightGBM model.

In [8]:
from src.plot import plot_one_sample

plot_one_sample(
    features=X_test,
    targets=y_test,
    example_id=2979,
    predictions=pd.Series(predictions)
)

In [9]:
plot_one_sample(
    features=X_test,
    targets=y_test,
    example_id=3979,
    predictions=pd.Series(predictions)
)